In [12]:
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image, ImageDraw, ImageOps
import PIL
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

try:
    os.mkdir('data')
except:
    print('Path Already Exists')

width, height = 400, 400

# Modern color scheme
BG_COLOR = "#1e1e2e"
CANVAS_BG = "#ffffff"
PRIMARY_COLOR = "#89b4fa"
SUCCESS_COLOR = "#a6e3a1"
WARNING_COLOR = "#f9e2af"
DANGER_COLOR = "#f38ba8"
TEXT_COLOR = "#cdd6f4"
ACCENT_COLOR = "#b4befe"

win = tk.Tk()
win.title("Handwritten Character Recognition")
win.configure(bg=BG_COLOR)
win.resizable(False, False)

# Modern fonts
font_btn = ('Segoe UI', 10, 'bold')
font_label = ('Segoe UI', 12, 'bold')
font_title = ('Segoe UI', 14, 'bold')

count = 0
pencil_size = 15  # Default pencil size

import joblib

clsfr = joblib.load('sinhala-character-knn.sav')

label_dict = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

def event_function(event):
    x = event.x
    y = event.y
    
    x1 = x - pencil_size
    y1 = y - pencil_size
    x2 = x + pencil_size
    y2 = y + pencil_size
    
    canvas.create_oval((x1, y1, x2, y2), fill='black', outline='black')
    img_draw.ellipse((x1, y1, x2, y2), fill='black')

def update_pencil_size(val):
    global pencil_size
    pencil_size = int(float(val))
    pencil_size_label.config(text=f'Pencil Size: {pencil_size}px')

def clear():
    global img, img_draw

    canvas.delete('all')
    img = Image.new('RGB', (width, height), (255, 255, 255))
    img_draw = ImageDraw.Draw(img)
    label_status.config(text='Draw a character to predict', fg=TEXT_COLOR)

def predict():
    img_array = np.array(img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    img_array = cv2.bitwise_not(img_array)
    
    coords = cv2.findNonZero(img_array)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        padding = 20
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(img_array.shape[1] - x, w + 2 * padding)
        h = min(img_array.shape[0] - y, h + 2 * padding)
        img_array = img_array[y:y+h, x:x+w]
    
    img_array = cv2.resize(img_array, (28, 28))
    img_array = np.reshape(img_array, (1, 784))

    result = clsfr.predict(img_array)[0]
    label = label_dict.get(str(result), result)
    label_status.config(text=f'Predicted: {label}', fg=PRIMARY_COLOR)

# Main container frame
main_frame = tk.Frame(win, bg=BG_COLOR, padx=15, pady=15)
main_frame.pack()

# Title
title_label = tk.Label(main_frame, text="✍ Handwritten Character Recognition", 
                       bg=BG_COLOR, fg=ACCENT_COLOR, font=font_title)
title_label.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Canvas with border
canvas_frame = tk.Frame(main_frame, bg=PRIMARY_COLOR, padx=3, pady=3)
canvas_frame.grid(row=1, column=0, columnspan=4, pady=(0, 10))

canvas = tk.Canvas(canvas_frame, width=width, height=height, bg=CANVAS_BG, 
                  highlightthickness=0, cursor="pencil")
canvas.pack()

# Pencil size slider frame
slider_frame = tk.Frame(main_frame, bg=BG_COLOR)
slider_frame.grid(row=2, column=0, columnspan=4, pady=(0, 10))

pencil_size_label = tk.Label(slider_frame, text=f'Pencil Size: {pencil_size}px', 
                             bg=BG_COLOR, fg=TEXT_COLOR, font=('Segoe UI', 9, 'bold'))
pencil_size_label.pack(pady=(0, 3))

pencil_slider = tk.Scale(slider_frame, from_=5, to=50, orient=tk.HORIZONTAL, 
                        command=update_pencil_size, bg=BG_COLOR, fg=TEXT_COLOR,
                        highlightthickness=0, length=250, troughcolor=ACCENT_COLOR,
                        activebackground=PRIMARY_COLOR, cursor='hand2')
pencil_slider.set(pencil_size)
pencil_slider.pack()

# Button frame
button_frame = tk.Frame(main_frame, bg=BG_COLOR)
button_frame.grid(row=3, column=0, columnspan=4, pady=(0, 10))

# Modern styled buttons
button_predict = tk.Button(button_frame, text='🔍 PREDICT', bg=PRIMARY_COLOR, fg='#000000', 
                          font=font_btn, command=predict, relief=tk.FLAT, padx=25, pady=8,
                          activebackground='#7aa2f7', cursor='hand2')
button_predict.grid(row=0, column=0, padx=8)

button_clear = tk.Button(button_frame, text='🗑️ CLEAR', bg=WARNING_COLOR, fg='#000000', 
                        font=font_btn, command=clear, relief=tk.FLAT, padx=25, pady=8,
                        activebackground='#e7d09f', cursor='hand2')
button_clear.grid(row=0, column=1, padx=8)

# Status label
status_frame = tk.Frame(main_frame, bg=BG_COLOR, pady=5)
status_frame.grid(row=4, column=0, columnspan=4)

label_status = tk.Label(status_frame, text='Draw a character to predict', 
                       bg=BG_COLOR, fg=TEXT_COLOR, font=font_label)
label_status.pack()

# Bind events
canvas.bind('<B1-Motion>', event_function)
img = Image.new('RGB', (width, height), (255, 255, 255))
img_draw = ImageDraw.Draw(img)

# Center window on screen
win.update_idletasks()
window_width = win.winfo_width()
window_height = win.winfo_height()
screen_width = win.winfo_screenwidth()
screen_height = win.winfo_screenheight()
x = (screen_width // 2) - (window_width // 2)
y = (screen_height // 2) - (window_height // 2)
win.geometry(f'+{x}+{y}')

win.mainloop()


Path Already Exists
